In [1]:
! git clone https://github.com/Mandalor-09/Phishing.git

Cloning into 'Phishing'...
remote: Enumerating objects: 373, done.
remote: Counting objects: 100% (373/373), done.
remote: Compressing objects: 100% (229/229), done.
remote: Total 373 (delta 210), reused 294 (delta 131), pack-reused 0
Receiving objects: 100% (373/373), 25.31 MiB | 13.09 MiB/s, done.
Resolving deltas: 100% (210/210), done.


In [2]:
%cd /content/Phishing

/content/Phishing


In [3]:
%pwd

'/content/Phishing'

In [4]:
!pip install -r requirements.txt

  Running setup.py develop for Phishing-Website-Detection


In [17]:
!python src/pipeline/training_pipeline.py

Smoting Completed Succesfully The Vale counts are 1    58000
0    58000
Name: phishing, dtype: int64
Trainind Data
Accuracy of XGBClassifier using HyperOPT: 0.9075632183908046
Precision: 0.8778734413436254
Recall: 0.9477335164835164
F2 Score: 0.9328859362887378
Test Data Loaded Successfully
PCA Model loaded Successfully
Standardization Model loaded Successfully
Testing Data
Accuracy of XGBClassifier using HyperOPT: 0.9105862068965517
Precision: 0.8790484194194841
Recall: 0.9495810055865922
F2 Score: 0.9345832932411443
Pipeline trained and saved to: artifacts/model/model.joblib


In [18]:
from sklearn.metrics import accuracy_score,precision_score, fbeta_score, confusion_matrix
import pandas as pd
import joblib
df3 = pd.read_csv('/content/Phishing/data/dataset_small.csv')
df3 = df3[list(pd.read_csv('/content/Phishing/artifacts/cleaned_data/train_data.csv').columns)]
X2 = df3.drop(columns='phishing',axis=1)
Y2 = df3['phishing']
pca = joblib.load('/content/Phishing/artifacts/components/pca.joblib')
X2 = pca.fit_transform(X2)
scaler = joblib.load('/content/Phishing/artifacts/components/standard.joblib')
X2 = scaler.transform(X2)
xgb_hyp = joblib.load('/content/Phishing/artifacts/model/model.joblib')
ypred2 = xgb_hyp.predict(X2)
accuracy = accuracy_score(Y2,ypred2)
print('Accuracy  ' ,accuracy)
y_true = Y2
y_pred = ypred2

# Assuming y_true contains the true labels and y_pred contains the predicted labels
precision = precision_score(y_true, y_pred)
conf_matrix = confusion_matrix(y_true, y_pred)
recall = conf_matrix[1, 1] / (conf_matrix[1, 1] + conf_matrix[1, 0])
f2_score = fbeta_score(y_true, y_pred, beta=2)

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F2 Score: {f2_score}')

Accuracy   0.8525705516241794
Precision: 0.833601213040182
Recall: 0.8969230267236598
F2 Score: 0.8835005753296091


In [19]:
import numpy as np
from urllib.parse import urlparse, parse_qs
import joblib
import socket

def is_ip_address(domain):
    try:
        socket.inet_aton(domain)
        return True
    except socket.error:
        return False

def extract_additional_url_features(url):
    parsed_url = urlparse(url)

    return {
        'qty_dot_url': url.count('.'),
        'qty_hyphen_url': url.count('-'),
        'qty_underline_url': url.count('_'),
        'qty_slash_url': url.count('/'),
        'qty_questionmark_url': url.count('?'),
        'qty_equal_url': url.count('='),
        'qty_at_url': url.count('@'),
        'qty_exclamation_url': url.count('!'),
        'qty_space_url': url.count(' '),
        'qty_tilde_url': url.count('~'),
        'qty_comma_url': url.count(','),
        'qty_plus_url': url.count('+'),
        'qty_asterisk_url': url.count('*'),
        'qty_hashtag_url': url.count('#'),
        'qty_dollar_url': url.count('$'),
        'qty_percent_url': url.count('%'),
        'qty_tld_url': len(parsed_url.netloc.split('.')[-1]),
        'length_url': len(url)
    }

def extract_additional_domain_features(url):
    # Parse the URL to get the domain
    domain = urlparse(url).netloc

    if not domain:
        return {
            'qty_dot_domain': -1,
            'qty_hyphen_domain': -1,
            'qty_underline_domain': -1,
            'qty_at_domain': -1,
            'qty_vowels_domain': -1,
            'domain_in_ip': -1,
            'server_client_domain': -1
        }

    return {
        'qty_dot_domain': domain.count('.'),
        'qty_hyphen_domain': domain.count('-'),
        'qty_underline_domain': domain.count('_'),
        'qty_at_domain': domain.count('@'),
        'qty_vowels_domain': sum(1 for char in domain if char.lower() in "aeiou"),
        'domain_in_ip': 1 if is_ip_address(domain) else 0,
        'server_client_domain': 1 if domain.startswith("www.") else 0
    }

def extract_additional_path_features(url):
    # Parse the URL to get the path
    path = urlparse(url).path

    if not path:
        return {
            'qty_dot_directory': -1,
            'qty_hyphen_directory': -1,
            'qty_underline_directory': -1,
            'qty_percent_directory': -1,
            'directory_length': -1
        }

    return {
        'qty_dot_directory': path.count('.'),
        'qty_hyphen_directory': path.count('-'),
        'qty_underline_directory': path.count('_'),
        'qty_percent_directory': path.count('%'),
        'directory_length': len(path)
    }

def extract_file_features(url):
    # Parse the URL to get the path
    path = urlparse(url).path

    if not path:
        return {
            'file_length': -1
        }

    # Extract the file name from the path
    file_name = path.split('/')[-1]

    # Attribute: Length of the file name
    file_length = len(file_name)

    return {
        'file_length': file_length
    }

def extract_additional_params_features(url):
    # Parse the URL to get the query parameters
    query_params = urlparse(url).query

    if not query_params:
        return {
            'qty_dot_params': -1,
            'qty_hyphen_params': -1,
            'qty_underline_params': -1,
            'qty_slash_params': -1,
            'qty_questionmark_params': -1,
            'qty_percent_params': -1
        }

    # Extract parameter names from the query string
    param_names = parse_qs(query_params).keys()

    return {
        'qty_dot_params': sum(param.count('.') for param in param_names),
        'qty_hyphen_params': sum(param.count('-') for param in param_names),
        'qty_underline_params': sum(param.count('_') for param in param_names),
        'qty_slash_params': sum(param.count('/') for param in param_names),
        'qty_questionmark_params': sum(param.count('?') for param in param_names),
        'qty_percent_params': sum(param.count('%') for param in param_names)
    }

def email_urlshorten(url):
    # Parse the URL
    parsed_url = urlparse(url)

    # Extract the domain from the URL
    domain = parsed_url.netloc

    if not domain:
        return {
            'email_in_url': -1,
            'tls_ssl_certificate' : -1,
            'url_shortened': -1
        }

    return {
        'email_in_url': 1 if '@' in url else 0,
        'tls_ssl_certificate' : 1 if url.startswith("https://") else 0,
        'url_shortened': 1 if domain in ['bit.ly', 'goo.gl', 'tinyurl.com', 'ow.ly'] else 0
    }

def extract_all_features(url):
    # Extract URL-based features
    url_features = extract_additional_url_features(url)

    # Extract Domain-based features
    domain_features = extract_additional_domain_features(url)

    # Extract Page-based features
    path_features = extract_additional_path_features(url)

    # Extract File-based feature
    file_feature = extract_file_features(url)

    # Extract Params-based features
    params_features = extract_additional_params_features(url)

    # Extract Additional Features
    additional_features = email_urlshorten(url)

    # Combine all features
    all_features = {**url_features, **domain_features, **path_features, **file_feature, **params_features, **additional_features}

    return all_features


In [21]:
url = "http://app.validchk.com/visitqr.aspx?vid=1073653"
# http://app.validchk.com/visitqr.aspx?vid=1073653
extracted_features = extract_all_features(url)

# Extract features and reshape into a 2D array
data = np.array(list(extracted_features.values())).reshape(1, -1)

# Assuming you have a PCA object
pca_transformed_data = pca.transform(data)

# Assuming you have a scaler object
scaled_data = scaler.transform(pca_transformed_data)


# Use the trained XGBBoost for prediction
prediction = xgb_hyp.predict(scaled_data)
print(prediction)

[1]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


In [22]:
import shutil
# Directory to zip (modify this based on your directory structure)
directory_to_zip = '/content/Phishing'

# Create a zip file
zip_file_path = '/content/Phishing.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', directory_to_zip)


'/content/Phishing.zip'